## Reproduce Published results with Starfish

This notebook walks through a workflow that reproduces a MERFISH result for one field of view using the starfish package.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib notebook

import pprint

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from showit import image as show_image
from starfish.constants import Indices, Features
from starfish.experiment import Experiment

In [ ]:
# load the data from cloudfront
experiment = Experiment()
experiment.read('https://dmf0bdeheu4zf.cloudfront.net/20180802/MERFISH/fov_001/experiment.json')

Individual imaging rounds and channels can also be visualized

In [ ]:
primary_image = experiment.image

In [ ]:
# show all imaging rounds of channel 0
primary_image.show_stack({Indices.CH: 0})

## Show input file format that specifies how the tiff stack is organized

The stack contains multiple images corresponding to the channel and imaging rounds. MERFISH builds a 16 bit barcode from 8 imaging rounds, each of which measures two channels that correspond to contiguous (but not necessarily consistently ordered) bits of the barcode. 

The MERFISH computational pipeline also constructs a scalar that corrects for intensity differences across each of the 16 images, e.g., one scale factor per bit position.

The stacks in this example are pre-registered using fiduciary beads. 

In [ ]:
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(experiment.format_metadata)

## Visualize codebook

The MERFISH codebook maps each barcode to a gene (or blank) feature. The codes in the MERFISH codebook are constructed from a 4-hamming error correcting code with exactly 4 "on" bits per barcode

In [ ]:
from starfish import Codebook
codebook = Codebook.from_json('https://dmf0bdeheu4zf.cloudfront.net/20180802/MERFISH/codebook.json')
codebook

## Filter and scale raw data before decoding

Begin filtering with a high pass filter to remove background signal.

In [ ]:
from starfish.image import Filter
ghp = Filter.GaussianHighPass(sigma=3)
ghp.run(primary_image, verbose=True)

The below algorithm deconvolves out the point spread function introduced by the microcope and is specifically designed for this use case. The number of iterations is an important parameter that needs careful optimization. 

In [ ]:
dpsf = Filter.DeconvolvePSF(num_iter=15, sigma=2)
dpsf.run(primary_image, verbose=True)

Recall that the image is pre-registered, as stated above. Despite this, individual RNA molecules may still not be perfectly aligned across imaging rounds. This is crucial in order to read out a measure of the itended barcode (across imaging rounds) in order to map it to the codebook. To solve for potential mis-alignment, the images can be blurred with a 1-pixel Gaussian kernel. The risk here is that this will obfuscate signals from nearby molecules. 

A local search in pixel space across imaging rounds can also solve this. 

In [ ]:
glp = Filter.GaussianLowPass(sigma=1, verbose=True)
glp.run(primary_image)

Use MERFISH-calculated size factors to scale the channels across the imaging rounds and visualize the resulting filtered and scaled images. Right now we have to extract this information from the metadata and apply this transformation manually.

In [ ]:
# assert a small portion of the tensor remains unchanged to check for modifications to the filters
expected = np.array(
    [[14, 18, 15,  9,  4,  2,  2,  3,  6, 14],
     [22, 25, 18,  8,  2,  0,  0,  1,  2,  6],
     [24, 26, 16,  6,  1,  0,  0,  0,  0,  2],
     [18, 18, 10,  3,  0,  0,  0,  0,  0,  1],
     [ 9,  8,  4,  1,  0,  0,  0,  0,  2,  3],
     [ 5,  4,  2,  0,  0,  0,  1,  4,  8, 11],
     [ 8,  6,  3,  1,  0,  1,  4, 11, 20, 26],
     [20, 17, 10,  4,  2,  2,  6, 16, 27, 34],
     [37, 33, 20,  9,  4,  4,  7, 13, 21, 25],
     [48, 44, 27, 13,  6,  5,  5,  7, 10, 11]], 
)
assert np.array_equal(primary_image.numpy_array[5, 1, 0, 1000:1010, 1000:1010], expected)

In [ ]:
scale_factors = {
    (t[Indices.ROUND], t[Indices.CH]): t['scale_factor'] 
    for index, t in primary_image.tile_metadata.iterrows()
}

In [ ]:
# it's important to convert the data to float here to retain the correct precision for the scaling. Later, starfish
# will operate entirely on float data and this cast can be removed
primary_image._data = primary_image._data.astype(np.float64)

# this is a scaling method. It would be great to use image.apply here. It's possible, but we need to expose H & C to 
# at least we can do it with get_slice and set_slice right now.

for indices in primary_image._iter_indices():
    data = primary_image.get_slice(indices)[0]
    scaled = data / scale_factors[indices[Indices.ROUND.value], indices[Indices.CH.value]]
    primary_image.set_slice(indices, scaled)

In [ ]:
# assert on a small part of the scaled image to verify values
expected = np.array(
    [[0.33527313, 0.43106545, 0.35922121, 0.21553273, 0.09579232,
      0.04789616, 0.04789616, 0.07184424, 0.14368848, 0.33527313],
     [0.52685777, 0.59870201, 0.43106545, 0.19158464, 0.04789616,
      0.        , 0.        , 0.02394808, 0.04789616, 0.14368848],
     [0.57475393, 0.62265009, 0.38316929, 0.14368848, 0.02394808,
      0.        , 0.        , 0.        , 0.        , 0.04789616],
     [0.43106545, 0.43106545, 0.23948081, 0.07184424, 0.        ,
      0.        , 0.        , 0.        , 0.        , 0.02394808],
     [0.21553273, 0.19158464, 0.09579232, 0.02394808, 0.        ,
      0.        , 0.        , 0.        , 0.04789616, 0.07184424]]
)
assert np.allclose(primary_image.numpy_array[5, 1, 0, 1000:1005, 1000:1010], expected)

In [ ]:
from scipy.stats import scoreatpercentile

In [ ]:
mp = primary_image.max_proj(Indices.ROUND, Indices.CH, Indices.Z)
clim = scoreatpercentile(mp, [0.5, 99.5])
show_image(mp, clim=clim)

## Use spot-detector to create 'encoder' table  for standardized input  to decoder

Each pipeline exposes a spot detector, and this spot detector translates the filtered image into an encoded table by detecting spots. The table contains the spot_id, the corresponding intensity (v) and the channel (c), imaging round (r) of each spot. 

The MERFISH pipeline merges these two steps together by finding pixel-based features, and then later collapsing these into spots and filtering out undesirable (non-spot) features. 

Therefore, no encoder table is generated, but a robust SpotAttribute and DecodedTable are both produced:

## Decode

Each assay type also exposes a decoder. A decoder translates each spot (spot_id) in the encoded table into a gene that matches a barcode in the codebook. The goal is to decode and output a quality score, per spot, that describes the confidence in the decoding. Recall that in the MERFISH pipeline, each 'spot' is actually a 16 dimensional vector, one per pixel in the image. From here on, we will refer to these as pixel vectors. Once these pixel vectors are decoded into gene values, contiguous pixels that are decoded to the same gene are labeled as 'spots' via a connected components labeler. We shall refer to the latter as spots.

There are hard and soft decodings -- hard decoding is just looking for the max value in the code book. Soft decoding, by contrast, finds the closest code by distance in intensity. Because different assays each have their own intensities and error modes, we leave decoders as user-defined functions. 

For MERFISH, which uses soft decoding, there are several parameters which are important to determining the result of the decoding method: 

### Distance threshold
In MERFISH, each pixel vector is a 16d vector that we want to map onto a barcode via minimum euclidean distance. Each barcode in the codebook, and each pixel vector is first mapped to the unit sphere by L2 normalization. As such, the maximum distance between a pixel vector and the nearest single-bit error barcode is 0.5176. As such, the decoder only accepts pixel vectors that are below this distance for assignment to a codeword in the codebook. 

### Magnitude threshold
This is a signal floor for decoding. Pixel vectors with an L2 norm below this floor are not considered for decoding. 

### Area threshold
Contiguous pixels that decode to the same gene are called as spots via connected components labeling. The minimum area of these spots are set by this parameter. The intuition is that pixel vectors, that pass the distance and magnitude thresholds, shold probably not be trusted as genes as the mRNA transcript would be too small for them to be real. This parameter can be set based on microscope resolution and signal amplification strategy.

### Crop size 
The crop size crops the image by a number of pixels large enough to eliminate parts of the image that suffer from boundary effects from both signal aquisition (e.g., FOV overlap) and image processing. Here this value is 40.

Given these three thresholds, for each pixel vector, the decoder picks the closest code (minimum distance) that satisfies each of the above thresholds, where the distance is calculated between the code and a normalized intensity vector and throws away subsequent spots that are too small.

In [ ]:
from starfish.spots import SpotFinder
psd = SpotFinder.PixelSpotDetector(
    codebook=codebook,
    metric='euclidean',
    distance_threshold=0.5176,
    magnitude_threshold=1,
    min_area=2,
    max_area=np.inf,
    norm_order=2, 
    crop_size=(0, 40, 40)
)

spot_intensities, prop_results = psd.find(primary_image)
spot_intensities

In [ ]:
# verify detected number of spots
assert spot_intensities.shape == (37742, 2, 8)

## Compare to results from paper 

The below plot aggregates gene copy number across single cells in the field of view and compares the results to the published intensities in the MERFISH paper. 

To make this match perfectly, run deconvolution 15 times instead of 14. As presented below, STARFISH displays a lower detection rate.  

In [ ]:
bench = pd.read_csv('https://dmf0bdeheu4zf.cloudfront.net/MERFISH/benchmark_results.csv', 
                    dtype = {'barcode':object})

benchmark_counts = bench.groupby('gene')['gene'].count()
genes, counts = np.unique(spot_intensities[Features.AXIS][Features.TARGET], return_counts=True)
result_counts = pd.Series(counts, index=genes)

tmp = pd.concat([result_counts, benchmark_counts], join='inner', axis=1).values

r = np.corrcoef(tmp[:, 1], tmp[:, 0])[0, 1]
x = np.linspace(50, 2000)
f, ax = plt.subplots(figsize=(6, 6))
ax.scatter(tmp[:, 1], tmp[:, 0], 50, zorder=2)
ax.plot(x, x, '-k', zorder=1)

plt.xlabel('Gene copy number Benchmark')
plt.ylabel('Gene copy number Starfish')
plt.xscale('log')
plt.yscale('log')
plt.title(f'r = {r}');

In [ ]:
# assert that notebook results match published ones. 
assert r > 0.99

## Visualize results

This image applies a pseudo-color to each gene channel to visualize the position and size of all called spots in a subset of the test image

In [ ]:
area_lookup = lambda x: 0 if x == 0 else prop_results.region_properties[x - 1].area
vfunc = np.vectorize(area_lookup)
mask = np.squeeze(vfunc(prop_results.label_image))
show_image(np.squeeze(prop_results.decoded_image)*(mask > 2), cmap = 'nipy_spectral')